In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from dataSrc import CovidDataSrc
cds = CovidDataSrc()

Loading lk data
done
loading covid data
done


In [2]:
cd = cds._covid_ds
full_date_range = pd.date_range(start=cd.Meldedatum.min(), end=cd.Meldedatum.max())
full_range_zeros =  pd.DataFrame(data=0, index=full_date_range, columns=['Zero'])

In [3]:
df_by_lk_and_date = cd.groupby(['IdLandkreis', 'Meldedatum']).agg({'AnzahlFall': np.sum}).reset_index()
train_inputs_smooth = []
for lk in cd.IdLandkreis.unique():
    ds_train = df_by_lk_and_date.loc[df_by_lk_and_date.IdLandkreis==lk].copy()
    ds_train.Meldedatum = pd.to_datetime(ds_train.Meldedatum)
    ds_train = ds_train.set_index(['Meldedatum'])
    ds_train = pd.DataFrame(full_range_zeros.join(ds_train, how='left').AnzahlFall.fillna(0))
    
    ds_train['smooth'] = ds_train.AnzahlFall \
        .rolling(window=7, closed='both', win_type='gaussian').mean(std=3.5)
    ds_train['cum_sum'] = ds_train.AnzahlFall.cumsum()
    ds_train['smooth_cum_sum'] = ds_train.smooth.cumsum()
    train_inputs_smooth.append(
        ds_train.smooth_cum_sum.dropna()
    )

In [4]:
min_date = train_inputs_smooth[0].index.min()
max_date = train_inputs_smooth[0].index.max()

In [5]:
full_train_data_by_day = \
    [
        [ train_data[date] for train_data in train_inputs_smooth]
        for date in 
            pd.date_range(start=min_date, end=max_date)
    ]

In [9]:
expected_offset = 7

In [11]:
data_dim = [
    (full_train_data_by_day[idx], full_train_data_by_day[idx + expected_offset]) 
    for idx in range(len(full_train_data_by_day) - expected_offset )
]data_dim

In [109]:
data_dim = len(train_expected[0][0])
model = keras.Sequential([
    keras.layers.LSTM(data_dim, return_sequences=True),
    keras.layers.Dense(2 * data_dim),
    keras.layers.Dense(data_dim),

])
model.compile(loss='mean_squared_error')

In [114]:
model.fit(
    x=[ [[1], [0]] * data_dim],
    y=[ [1] * data_dim]
)

TypeError: 'NoneType' object is not callable

In [ ]:
model.fit(
    x=[ train[0] for train in train_expected],
    y=[ train[1] for train in train_expected]
)